In [1]:
from pymongo import MongoClient
from sklearn.model_selection import train_test_split
from keras.wrappers.scikit_learn import KerasRegressor
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import GaussianNoise
from keras.utils import plot_model
from keras.callbacks import TensorBoard
from keras.utils import vis_utils as vizu
from keras.layers import Dropout
from keras import callbacks
from keras import backend
from scipy.integrate import simps
from scipy.stats import f
from math import sqrt
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.manifold import TSNE
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
from sklearn.svm import SVR
from sklearn.neural_network import MLPRegressor
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import f_regression, mutual_info_regression
def knn_baseline_model():
    
    def root_mean_squared_error(y_true, y_pred):
        return backend.sqrt(backend.mean(backend.square(y_pred - y_true), axis=-1))
    
    # create model
    model = Sequential()
    #model.add(GaussianNoise(stddev=0.1, input_shape=(14,)))
    model.add(Dense(7, activation='relu', input_shape=(16,)))
    model.add(Dropout(0.5))
    model.add(GaussianNoise(0.1))
    model.add(Dense(24, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(1, kernel_initializer='normal'))
    # Compile model
    model.compile(loss=root_mean_squared_error, optimizer='adam')
    #model.compile(loss='mse', optimizer='adam')
    return model
def coeff_determination(y_true, y_pred):
    ssres = np.sum(np.square(y_true - y_pred))
    sstot = np.sum(np.square(y_true - np.mean(y_pred)))
    return 1 - ssres / sstot
def REC(y_true , y_pred):
    
    Accuracy = []
    Begin_Range = 0
    End_Range = 1.5
    Interval_Size = 0.01
    
    # List of epsilons
    Epsilon = np.arange(Begin_Range , End_Range , Interval_Size)
    
    # Main Loops
    for i in range(len(Epsilon)):
        count = 0.0
        for j in range(len(y_true)):
            y_true[j]=float(y_true[j])
            y_true[j]
            y_pred[j]
            
            np.linalg.norm(y_true[j] - y_pred[j])
            np.sqrt( np.linalg.norm(y_true[j]) **2 + np.linalg.norm(y_pred[j])**2 ) < Epsilon[i]
            
            if np.linalg.norm(y_true[j] - y_pred[j]) / np.sqrt( np.linalg.norm(y_true[j]) **2 + np.linalg.norm(y_pred[j])**2 ) < Epsilon[i]:
                count = count + 1
        
        Accuracy.append(count/len(y_true))
    
    # Calculating Area Under Curve using Simpson's rule
    AUC = simps(Accuracy , Epsilon ) / End_Range
        
    # returning epsilon , accuracy , area under curve    
    return Epsilon, Accuracy, AUC
def model_evaluate (model, X_train, X_test, y_train, y_test):

    y_pred = model.predict(X_test)
    Deviation, Accuracy, auc = REC(y_test.values, y_pred)
    rmse = sqrt(mean_squared_error(y_test, y_pred))
    #r2 = r2_score(y_test, y_pred)
    r2 = coeff_determination(y_test, y_pred)
    mape = ((y_test - y_pred) / y_test).abs().mean()*100
    
    f_test = np.var(y_test) / np.var(y_pred)
    df1 = len(y_test) - 1
    df2 = len(y_pred) - 1
    p_value = 1 - 2 * abs(0.5 - f.cdf(f_test, df1, df2))
    
    p_stars = ''
    if p_value <= 0.05:
        p_stars = '*'
    if p_value <= 0.01:
        p_stars = '**'
    if p_value <= 0.001:
        p_stars = '***'
    #print(F, p_value)
    
    y_validate = model.predict(X_train)
    training_loss = sqrt(mean_squared_error(y_train, y_validate))
    
    return_dict = {'rmse':rmse, 
                   'mape':mape, 
                   'r2':r2, 
                   'auc':auc,
                   'training_loss':training_loss, 
                   'f_test':f_test,
                   'p':p_stars,
                   'y_pred':y_pred, 
                   'y_validate':y_validate}
    
    return return_dict

C:\Users\user\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
sp105 = pd.read_csv('1051SP_w9.csv')
sp105.fillna(0.0, inplace=True)
sp105 = sp105.drop(['name'], axis=1)
sp105 = sp105.drop(['username'], axis=1)
sp105 = sp105.drop(['final_score'], axis=1)
sp105 = sp105.drop(['label_2'], axis=1)
sp105.head(3)

,video_watching_days,video_watching_times,video_watching_total_time,bbs_days,bbs_num,video_watching_num,video_time_between_start_first_watching,pre_watching_num,in_watching_num,re_watching_num,no_watching_num,post_watching_num,online_quiz,online_hw,pgm,quiz,adjust_score
0,10,33,1544,1,1,23,270,3,3,2,22,9,300,140,96,0,54
1,6,43,867,0,0,19,153,0,0,0,25,7,100,90,93,44,68
2,5,18,787,0,0,15,136,0,1,1,24,12,200,140,0,52,70


In [3]:
data=sp105
data.to_csv('cycu sp105.csv')
data_y= data['adjust_score']
data_x = data.drop(['adjust_score'], axis=1)

In [4]:
data

,video_watching_days,video_watching_times,video_watching_total_time,bbs_days,bbs_num,video_watching_num,video_time_between_start_first_watching,pre_watching_num,in_watching_num,re_watching_num,no_watching_num,post_watching_num,online_quiz,online_hw,pgm,quiz,adjust_score
0,10,33,1544,1,1,23,270,3,3,2,22,9,300,140,96,0,54
1,6,43,867,0,0,19,153,0,0,0,25,7,100,90,93,44,68
2,5,18,787,0,0,15,136,0,1,1,24,12,200,140,0,52,70
3,10,57,533,6,20,26,200,3,5,2,20,8,200,40,65,44,66
4,8,64,1944,0,0,41,240,0,1,1,24,16,300,90,97,44,50
5,1,2,2,1,1,1,52,0,0,0,25,0,100,200,97,64,74
6,12,27,249,6,21,17,158,2,1,2,24,9,300,260,66,44,60
7,1,1,0,1,1,0,0,0,0,0,25,0,0,0,0,40,17
8,18,88,3111,7,26,58,333,19,3,2,22,7,300,120,80,44,82
9,9,47,974,3,35,35,97,19,8,1,17,1,300,120,62,64,62


In [5]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data_x, data_y, test_size=0.3)

In [6]:
mlp = KerasRegressor(build_fn=knn_baseline_model, epochs=1000, batch_size=2, verbose=0)

In [7]:
mlp.fit(X_train, y_train) 
result_dict = model_evaluate (mlp, X_train, X_test, y_train, y_test)
print(result_dict)

{'rmse': 24.60407619808748, 'mape': 38.38096644681773, 'r2': 0.390457750705966, 'auc': 0.8067479674796747, 'training_loss': 24.845484069838843, 'f_test': 6.32674869354357, 'p': '***', 'y_pred': array([56.37134 , 52.416786, 40.721836, 19.492668, 39.52856 , 39.3787  ,
       49.677208, 21.330471, 31.892944, 33.378654, 38.79115 , 40.19919 ,
       51.02875 , 31.218994, 20.501118, 49.459003, 25.260319, 56.108166,
       49.81375 , 31.309559, 42.183872, 56.56055 , 37.16904 , 39.24611 ,
       29.812273, 30.348152, 46.396107, 27.93114 , 28.933113, 34.313065,
       39.313774, 46.334454, 22.792587, 26.749409, 29.54161 , 21.385239,
       49.78546 , 46.59951 , 33.736256, 43.36121 , 48.20075 ],
      dtype=float32), 'y_validate': array([55.14153 , 61.48422 , 62.315113, 40.777008, 53.038227, 35.966785,
       29.694805, 44.5358  , 39.211246, 47.031036, 59.016506, 44.973686,
       38.16896 , 37.67213 , 18.33181 , 53.38927 , 41.336693, 38.98366 ,
       54.69415 , 36.7935  , 33.2686  , 42.836834,